In [13]:
from keras.models import Sequential
from keras.layers import Dense,RepeatVector,Dropout
from keras.layers import Flatten
from sklearn.preprocessing import MinMaxScaler
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pandas as pd
import numpy as np
import time
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

In [14]:
data = pd.read_csv('daily-website-visitors.csv')


In [15]:
data

,Row,Day,Day.Of.Week,Date,Page.Loads,Unique.Visits,First.Time.Visits,Returning.Visits
0,1,Sunday,1,9/14/2014,2146,1582,1430,152
1,2,Monday,2,9/15/2014,3621,2528,2297,231
2,3,Tuesday,3,9/16/2014,3698,2630,2352,278
3,4,Wednesday,4,9/17/2014,3667,2614,2327,287
4,5,Thursday,5,9/18/2014,3316,2366,2130,236
...,...,...,...,...,...,...,...,...
2162,2163,Saturday,7,8/15/2020,2221,1696,1373,323
2163,2164,Sunday,1,8/16/2020,2724,2037,1686,351
2164,2165,Monday,2,8/17/2020,3456,2638,2181,457
2165,2166,Tuesday,3,8/18/2020,3581,2683,2184,499


In [16]:
data = data.filter(['Page.Loads'])


In [17]:
# Lấy 100 dòng cuối cùng của df
data = data.tail(1000)
data

,Page.Loads
1167,2658
1168,2391
1169,3202
1170,4349
1171,4886
...,...
2162,2221
2163,2724
2164,3456
2165,3581


In [18]:
#Tiền xử lý
data.fillna(0, inplace=True)

# Normalize data
scaler = MinMaxScaler()
dataScale = scaler.fit_transform(data.values.reshape(-1,1))

In [19]:
#Chia dữ liệu thành train test
data = pd.DataFrame(dataScale)

#dataScale = data
train_percent = 0.7
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [20]:

data = train_data

#dataScale = data
train_percent = 0.7
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]


In [21]:
#Chuẩn bị dữ liệu để đưa vào mô hình
def prepare_data(data, size_window,size_predict,stepWindow):
    X, y = [], []
    startWindow = 0
    for i in range(len(data) - size_window - 1):
        if (len(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0]) != size_predict):
            break
        X.append(data[startWindow:(startWindow + size_window), :])
        y.append(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0])
          
       
        startWindow += stepWindow
    return np.array(X), np.array(y)


size_window = 7
size_predict = 1
stepWindow = 1
X_train, y_train = prepare_data(train_data.values, size_window, size_predict, stepWindow)
X_test, y_test = prepare_data(test_data.values, size_window, size_predict, stepWindow)

In [22]:
start_time = time.time()

# Tạo một list chứa các số không phải là số lucas
number_list = [5,6,8,9]

# Chọn số lượng hidden layer sử dụng theo số trên
num_hidden_layer = number_list[0] # ví dụ chọn số 5

In [23]:

# Tạo mô hình CNN
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(size_window, 1)))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))

model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
for i in range(num_hidden_layer): #Thêm num_hidden_layer lớp ẩn với 64 đơn vị mỗi lớp
    model.add(Dense(units=100, activation='relu')) #Tham số chỉ định số lượng tế bào thần kinh trong lớp
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

# Compile mô hình với Adam optimizer và Mean Squared Error loss
model.compile(loss='mse', optimizer='adam')


In [24]:
history = model.fit(X_train, y_train, epochs=9000, batch_size=16)

Epoch 1/10000
40/40 [==============================] - 2s 2ms/step - loss: 0.1317
Epoch 2/10000
40/40 [==============================] - 0s 2ms/step - loss: 0.0466
Epoch 3/10000
40/40 [==============================] - 0s 2ms/step - loss: 0.0450
Epoch 4/10000
40/40 [==============================] - 0s 2ms/step - loss: 0.0312
Epoch 5/10000
40/40 [==============================] - 0s 2ms/step - loss: 0.0194
Epoch 6/10000
40/40 [==============================] - 0s 2ms/step - loss: 0.0172
Epoch 7/10000
40/40 [==============================] - 0s 2ms/step - loss: 0.0150
Epoch 8/10000
40/40 [==============================] - 0s 2ms/step - loss: 0.0149
Epoch 9/10000
40/40 [==============================] - 0s 2ms/step - loss: 0.0139
Epoch 10/10000
40/40 [==============================] - 0s 2ms/step - loss: 0.0137
Epoch 11/10000
40/40 [==============================] - 0s 2ms/step - loss: 0.0102
Epoch 12/10000
40/40 [==============================] - 0s 2ms/step - loss: 0.0115
Epoch 13/1000

KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(X_test)
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.reshape(y_pred, (-1, 1))
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred)

print('MSE: ', mse)
print('RMSE: ', rmse)
print('MAPE: ', mape)

In [ ]:
len(data)